<a href="https://colab.research.google.com/github/ashhwiithac22/Deep-Learning-Experiments/blob/main/DL_Ex6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch #the deep learning library used to train neural networks
import torch.nn as nn # Imports neural network
import torch.optim as optim #used to update model weights during training
from torchvision import datasets, transforms # Imports ready-made datasets (like MNIST, CIFAR)
import cv2 # Imports OpenCV library used for image processing
import numpy as np

class HistEq:
    def __call__(self, img):
        img = np.array(img) #converts the image to numpy array
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.equalizeHist(img) #Applies histogram equalization to improve image contrast
        img = torch.tensor(img).unsqueeze(0).float() / 255.0 # Convert image to tensor, add channel dimension, convert to float, and normalize pixel values
        return img

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 8, 3)    # 1  → input channels (grayscale image) # 8  → number of output  # 3  → kernel size (3x3)
        self.pool = nn.MaxPool2d(2,2)
        self.fc = nn.Linear(8*15*15, 10) #fully connected layer

    def forward(self, x):
        x = self.pool(torch.relu(self.conv(x)))
        x = x.view(x.size(0), -1) #1D vector
        return self.fc(x)

def train_test(loader):
    model = SimpleCNN()
    loss_fn = nn.CrossEntropyLoss() #loss function
    optimizer = optim.Adam(model.parameters(), lr=0.001) #updates weights with learning rate 0.001

    # Train (1 epoch)
    for images, labels in loader:
        optimizer.zero_grad() #clears old gradient
        outputs = model(images) #predicts output
        loss = loss_fn(outputs, labels)  # Calculates loss between predicted output and actual labels
        loss.backward()
        optimizer.step()  # Updates model weights
    correct = 0  #Variables to count correct predictions and total samples
    total = 0
    with torch.no_grad(): # Disables gradient calculation during testing
        for images, labels in loader:
            outputs = model(images)
            _, pred = torch.max(outputs, 1)
            correct += (pred == labels).sum().item() #counts number of correct predictions
            total += labels.size(0) #counts total number of images

    return correct / total

transform_normal = transforms.Compose([transforms.Grayscale(),transforms.ToTensor()]) # Converts image to grayscale and to  PyTorch tensor
transform_hist = transforms.Compose([HistEq()])

dataset_normal = datasets.CIFAR10(root='.', train=True, download=True, transform=transform_normal) #. -> dataset is downloaded in currently working directory
dataset_hist = datasets.CIFAR10(root='.', train=True, download=True, transform=transform_hist)

loader_normal = torch.utils.data.DataLoader(dataset_normal, batch_size=128, shuffle=True) #128 images
loader_hist = torch.utils.data.DataLoader(dataset_hist, batch_size=128, shuffle=True)

print("WITHOUT Histogram Equalization")
acc1 = train_test(loader_normal)
print("Accuracy:", acc1)

print("\nWITH Histogram Equalization")
acc2 = train_test(loader_hist)
print("Accuracy:", acc2)

print("\nObservation:")
if acc1 > acc2:
    print("Accuracy without histogram equalization is higher.")
else:
    print("Accuracy with histogram equalization is higher.")


100%|██████████| 170M/170M [00:10<00:00, 16.1MB/s]


WITHOUT Histogram Equalization
Accuracy: 0.38558

WITH Histogram Equalization
Accuracy: 0.41446

Observation:
Accuracy with histogram equalization is higher.
